In [2]:
# # !ls -lrt 

# # step1
# !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
# !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
# !apt-get update -qq 2>&1 > /dev/null
# !apt-get -y install -qq google-drive-ocamlfuse fuse
# from google.colab import auth
# auth.authenticate_user()
# from oauth2client.client import GoogleCredentials
# creds = GoogleCredentials.get_application_default()
# import getpass
# !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
# vcode = getpass.getpass()
# !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

# # step2
# # 指定Google Drive云端硬盘的根目录，名为drive
# !mkdir -p drive
# !google-drive-ocamlfuse drive

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131304 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.3-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [3]:
!ls -lrt 
import os
os.chdir('drive/Colab/github/')
# fetch_pretrained_inception_v3()
# !cp -r datasets drive/

total 12
drwxr-xr-x 1 root root 4096 Apr  4 20:20 sample_data
-rw-r--r-- 1 root root 2480 Apr 25 03:15 adc.json
drwxr-xr-x 2 root root 4096 Apr 25 03:16 drive


In [0]:
#20190420
# 更改为每行输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 
import sys
import os
os.chdir("/content")

import tarfile
from six.moves import urllib
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
FLOWERS_URL = "http://download.tensorflow.org/example_images/flower_photos.tgz"
FLOWERS_PATH = os.path.join("datasets", "flowers")
def download_progress(count, block_size, total_size):
    percent = count * block_size * 100 // total_size
    sys.stdout.write("\rDownloading: {}%".format(percent))
    sys.stdout.flush()
def fetch_flowers(url=FLOWERS_URL, path=FLOWERS_PATH):
    if os.path.exists(FLOWERS_PATH):
        return
    os.makedirs(path, exist_ok=True)
    tgz_path = os.path.join(path, "flower_photos.tgz")
    urllib.request.urlretrieve(url, tgz_path, reporthook=download_progress)
    flowers_tgz = tarfile.open(tgz_path)
    flowers_tgz.extractall(path=path)
    flowers_tgz.close()
    os.remove(tgz_path)

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)    
#to fetch the inception_v3 checkpoint 
TF_MODELS_URL = "http://download.tensorflow.org/models"
INCEPTION_V3_URL = TF_MODELS_URL + "/inception_v3_2016_08_28.tar.gz"
INCEPTION_PATH = os.path.join("datasets", "inception")

INCEPTION_V3_CHECKPOINT_PATH = os.path.join(INCEPTION_PATH, "inception_v3.ckpt")

def download_progress(count, block_size, total_size):
    percent = count * block_size * 100 // total_size
    sys.stdout.write("\rDownloading: {}%".format(percent))
    sys.stdout.flush()

def fetch_pretrained_inception_v3(url=INCEPTION_V3_URL, path=INCEPTION_PATH):
    if os.path.exists(INCEPTION_V3_CHECKPOINT_PATH):
        return
    os.makedirs(path, exist_ok=True)
    tgz_path = os.path.join(path, "inception_v3.tgz")
    print(tgz_path)
    urllib.request.urlretrieve(url, tgz_path, reporthook=download_progress)

    inception_tgz = tarfile.open(tgz_path)
    inception_tgz.extractall(path=path)
    inception_tgz.close()
    os.remove(tgz_path)    
print ("done")    
def get_model_params():
    gvars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
    return {gvar.op.name: value for gvar, value in zip(gvars, tf.get_default_session().run(gvars))}

done


In [0]:
fetch_flowers()
fetch_pretrained_inception_v3()
flowers_root_path = os.path.join(FLOWERS_PATH, "flower_photos")
flower_classes = sorted([dirname for dirname in os.listdir(flowers_root_path)
                  if os.path.isdir(os.path.join(flowers_root_path, dirname))])
flower_classes
#获得各类花的路径
from collections import defaultdict
#默认字典 当KEY不存在时，使用默认值 本例为空LIST
image_paths = defaultdict(list)

for flower_class in flower_classes:
    image_dir = os.path.join(flowers_root_path, flower_class)
    for filepath in os.listdir(image_dir):
        if filepath.endswith(".jpg"):
            image_paths[flower_class].append(os.path.join(image_dir, filepath))
for paths in image_paths.values():
    paths.sort()            
    

['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']

In [0]:
from scipy.misc import imresize

#from scipy.misc import skimage
#import skimage
#from skimage import transform

def prepare_image(image, target_width = 299, target_height = 299, max_zoom = 0.2):
    """Zooms and crops the image randomly for data augmentation."""

    # 首先，让我们找到最大的边界框，其中目标尺寸比率适合图像
    height = image.shape[0]
    width = image.shape[1]
    image_ratio = width / height
    target_image_ratio = target_width / target_height
    crop_vertically = image_ratio < target_image_ratio
    crop_width = width if crop_vertically else int(height * target_image_ratio)
    crop_height = int(width / target_image_ratio) if crop_vertically else height
        
    # 现在让我们用随机因子缩小这个边界框（将尺寸除以1.0到 1.0 +`max_zoom`之间的随机数。
    resize_factor = np.random.rand() * max_zoom + 1.0
    crop_width = int(crop_width / resize_factor)
    crop_height = int(crop_height / resize_factor)
    
    # 接下来，我们可以在图像上为此边界框选择一个随机位置。
    x0 = np.random.randint(0, width - crop_width)
    y0 = np.random.randint(0, height - crop_height)
    x1 = x0 + crop_width
    y1 = y0 + crop_height
    
    # 让我们使用我们构建的随机边界框裁剪图像。
    image = image[y0:y1, x0:x1]

    # 让我们也以50％的概率水平翻转图像：
    if np.random.rand() < 0.5:
        image = np.fliplr(image)

    # 现在，让我们将图像调整为目标尺寸。
    image = imresize(image, (target_width, target_height))
    
    # 最后，让我们确保颜色表示为32位浮点数，范围从0.0到1.0（现在）：
    return image.astype(np.float32) / 255

In [0]:
def prepare_image_with_tensorflow(image, target_width = 299, target_height = 299, max_zoom = 0.2):
    """Zooms and crops the image randomly for data augmentation."""
    """随机缩放和裁剪图像以进行数据增强"""
    
    # 首先，让我们找到最大的边界框，其中目标尺寸比率适合图像
    image_shape = tf.cast(tf.shape(image), tf.float32)
    height = image_shape[0]
    width = image_shape[1]
    image_ratio = width / height
    target_image_ratio = target_width / target_height
    crop_vertically = image_ratio < target_image_ratio
    crop_width = tf.cond(crop_vertically,
                         lambda: width,
                         lambda: height * target_image_ratio)
    crop_height = tf.cond(crop_vertically,
                          lambda: width / target_image_ratio,
                          lambda: height)

    # 现在让我们用随机因子缩小这个边界框（将尺寸除以1.0到 1.0 +`max_zoom`之间的随机数。
    # between 1.0 and 1.0 + `max_zoom`.
    resize_factor = tf.random_uniform(shape=[], minval=1.0, maxval=1.0 + max_zoom)
    crop_width = tf.cast(crop_width / resize_factor, tf.int32)
    crop_height = tf.cast(crop_height / resize_factor, tf.int32)
    box_size = tf.stack([crop_height, crop_width, 3])   # 3 = number of channels

    # 让我们使用我们计算的大小的随机边界框裁剪图像
    image = tf.random_crop(image, box_size)

    # 让我们也以50％的概率水平翻转图像：
    image = tf.image.random_flip_left_right(image)

    # resize_bilinear函数需要4D张量（一批图像），因此我们需要首先扩展维数：
    image_batch = tf.expand_dims(image, 0)

    # 最后，让我们将图像的大小调整为目标尺寸。 请注意，此函数返回float32张量。
    # returns a float32 tensor.
    image_batch = tf.image.resize_bilinear(image_batch, [target_height, target_width])
    image = image_batch[0] / 255  # back to a single image, and scale the colors from 0.0 to 1.0
    return image

In [0]:
# reset_graph()

# input_image = tf.placeholder(tf.uint8, shape=[None, None, 3])
# prepared_image_op = prepare_image_with_tensorflow(input_image)

# with tf.Session():
#     prepared_image = prepared_image_op.eval(feed_dict={input_image: example_image})
    
# plt.figure(figsize=(6, 6))
# plt.imshow(prepared_image)
# plt.title("{}x{}".format(prepared_image.shape[1], prepared_image.shape[0]))
# plt.axis("off")
# plt.show()

In [0]:
from tensorflow.contrib.slim.nets import inception
import tensorflow.contrib.slim as slim
reset_graph()

height = inception.inception_v3.default_image_size
width  = inception.inception_v3.default_image_size
channels =3
n_outputs = len(flower_classes)
X = tf.placeholder(tf.float32, shape=[None, height, width, channels], name="X")
training = tf.placeholder_with_default(False, shape=[])

with slim.arg_scope(inception.inception_v3_arg_scope()):
    logits, end_points = inception.inception_v3(X, num_classes=1001, is_training=training)
#     prelogits = tf.squeeze(end_points["PreLogits"], axis=[1, 2])
#     flower_logits = slim.fully_connected(prelogits, n_outputs, scope='flower_logits')
inception_saver = tf.train.Saver()

prelogits = tf.squeeze(end_points["PreLogits"], axis=[1, 2])
with tf.name_scope("new_output_layer"):
    flower_logits = tf.layers.dense(prelogits, n_outputs, name="flower_logits")
    Y_proba = tf.nn.softmax(flower_logits, name="Y_proba")

y = tf.placeholder(tf.int32, shape=[None])

with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=flower_logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    flower_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="flower_logits")
    training_op = optimizer.minimize(loss, var_list=flower_vars)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(flower_logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver() 

In [0]:
[v.name for v in flower_vars]

['flower_logits/kernel:0', 'flower_logits/bias:0']

In [0]:
flower_class_ids = {flower_class: index for index, flower_class in enumerate(flower_classes)}
flower_class_ids
flower_paths_and_classes = []
for flower_class, paths in list(image_paths.items()):
    for path in paths:
        flower_paths_and_classes.append((path, flower_class_ids[flower_class]))
valid_ratio =0.1
test_ratio = 0.1
train_size = int(len(flower_paths_and_classes) * (1 - test_ratio-valid_ratio))
valid_size = int(len(flower_paths_and_classes) * (valid_ratio))
                 
np.random.shuffle(flower_paths_and_classes)

flower_paths_and_classes_train = flower_paths_and_classes[:train_size]
flower_paths_and_classes_valid = flower_paths_and_classes[train_size:(train_size+valid_size)]     
flower_paths_and_classes_test = flower_paths_and_classes[(train_size+valid_size):]     
print(("train_ratio",float(1-valid_ratio-test_ratio),"valid_ratio:",valid_ratio," test_ratio:",test_ratio))
print(("train_size:",len(flower_paths_and_classes_train),"valid_size:",len(flower_paths_and_classes_valid),"test_size:",len(flower_paths_and_classes_test)))

{'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}

('train_ratio', 0.8, 'valid_ratio:', 0.1, ' test_ratio:', 0.1)
('train_size:', 2936, 'valid_size:', 367, 'test_size:', 367)


In [0]:
from random import sample
# from preprocessing import inception_preprocessing
import matplotlib.image as mpimg

def prepare_batch(flower_paths_and_classes, batch_size):
    if batch_size==-1:
        batch_paths_and_classes = flower_paths_and_classes
    else:
        batch_paths_and_classes = sample(flower_paths_and_classes, batch_size)
    images = [mpimg.imread(path)[:, :, :channels] for path, labels in batch_paths_and_classes]
#     print((images[0].shape))
    prepared_images = [prepare_image(image) for image in images]
#     print((prepared_images[0].shape))

    X_batch = 2 * np.stack(prepared_images) - 1 # 初始期望颜色范围从-1到1
    y_batch = np.array([labels for path, labels in batch_paths_and_classes], dtype=np.int32)
    return X_batch, y_batch


In [0]:
n_epochs = 100
batch_size = 64
n_iterations_per_epoch = len(flower_paths_and_classes_train) // batch_size
print("n_iterations_per_epoch",n_iterations_per_epoch)
INCEPTION_PATH = os.path.join("datasets", "inception")
INCEPTION_V3_CHECKPOINT_PATH = os.path.join(INCEPTION_PATH, "inception_v3.ckpt")

best_loss_val = np.infty
check_times =5
check_interval = int (n_iterations_per_epoch/check_times)
checks_since_last_progress = 0
max_checks_without_progress = 1000
best_model_params = None 
X_batch_valid, y_batch_valid = prepare_batch(flower_paths_and_classes_valid, batch_size=-1)
X_test, y_test = prepare_batch(flower_paths_and_classes_test, batch_size=-1)
with tf.Session() as sess:
    init.run()
    inception_saver.restore(sess, INCEPTION_V3_CHECKPOINT_PATH)
    
#     inception_saver.restore(sess, "./my_flowers_model")

    for epoch in range(n_epochs):
        print("Epoch", epoch)
        
        for iteration in range(n_iterations_per_epoch):
            X_batch, y_batch = prepare_batch(flower_paths_and_classes_train, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, training: True})
            if iteration %check_interval ==0:
                loss_val = loss.eval(feed_dict ={X: X_batch_valid, y: y_batch_valid})
                acc_val = accuracy.eval(feed_dict ={X: X_batch_valid, y: y_batch_valid})
                if loss_val < best_loss_val:
                  
                    best_loss_val = loss_val
                    checks_since_last_progress = 0
                    best_model_params = get_model_params()
                    save_path = saver.save(sess, "./my_flowers_model")
                    print ("better loss val!")
                else:
                    checks_since_last_progress += 1        
                print("Epoch {}, iteration: {}, acc_val: {:.4f}%,  valid.loss: {:.4f}, valid. best loss: {:.6f}".format(
                          epoch, iteration+1, acc_val*100,loss_val, best_loss_val))            
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_valid = accuracy.eval(feed_dict={X: X_batch_valid, y: y_batch_valid})
        print("Epoch {}, last batch accuracy: {:.4f}%, valid. accuracy: {:.4f}%, valid. best loss: {:.6f}".format(
                  epoch, acc_batch * 100, acc_valid * 100, best_loss_val))
        if checks_since_last_progress > max_checks_without_progress:
            print("Early stopping!")
            break        

#         save_path = saver.save(sess, "./my_flowers_model")

n_iterations_per_epoch 45


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


INFO:tensorflow:Restoring parameters from datasets/inception/inception_v3.ckpt
Epoch 0
better loss val!
Epoch 0, iteration: 1, acc_val: 13.8965%,  valid.loss: 1.7830, valid. best loss: 1.782961
better loss val!
Epoch 0, iteration: 10, acc_val: 41.9619%,  valid.loss: 1.4489, valid. best loss: 1.448888
better loss val!
Epoch 0, iteration: 19, acc_val: 44.4142%,  valid.loss: 1.2625, valid. best loss: 1.262461
better loss val!
Epoch 0, iteration: 28, acc_val: 51.2262%,  valid.loss: 1.1818, valid. best loss: 1.181785
better loss val!
Epoch 0, iteration: 37, acc_val: 50.9537%,  valid.loss: 1.1399, valid. best loss: 1.139948
Epoch 0, last batch accuracy: 57.8125%, valid. accuracy: 62.1253%, valid. best loss: 1.139948
Epoch 1
better loss val!
Epoch 1, iteration: 1, acc_val: 62.9428%,  valid.loss: 0.9471, valid. best loss: 0.947112
Epoch 1, iteration: 10, acc_val: 50.6812%,  valid.loss: 1.2118, valid. best loss: 0.947112
Epoch 1, iteration: 19, acc_val: 56.4033%,  valid.loss: 1.1180, valid. bes

In [0]:
!ls -lrt drive
!ls -lrt drive/Colab

In [0]:


n_test_batches = 10
X_test_batches = np.array_split(X_test, n_test_batches)
y_test_batches = np.array_split(y_test, n_test_batches)

with tf.Session() as sess:
    saver.restore(sess, "./my_flowers_model")

    print("Computing final accuracy on the test set (this will take a while)...")
    acc_test = np.mean([
        accuracy.eval(feed_dict={X: X_test_batch, y: y_test_batch})
        for X_test_batch, y_test_batch in zip(X_test_batches, y_test_batches)])
    print("Test accuracy:", acc_test)
    sess_path = os.path.join("drive/Colab","checkpoint")
    print (sess_path)
    if os.path.exists(sess_path):
        pass
    else:
      os.makedirs(sess_path)
    sess_path = os.path.join(sess_path,"my_flowers_newbest_model")
    saver.save(sess,sess_path)    

In [0]:
# # !os.chdir("drive/Colab Notebooks/dog_project") 
# import os
# !ls -lrt
# os.chdir("/content")
# !ls -lrt 
